# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
!pip install nltk
!pip install plotly
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vivi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vivi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Vivi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [49]:
# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

In [52]:
# load data from database
import os
path = 'sqlite:///' + 'DisasterResponse.db'
engine = create_engine(path)
df = pd.read_sql("disaster_messages", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
df_direct = df[df['genre'] == 'news']
categories_direct =  df_direct[df_direct.columns[4:]]
cate_counts_direct = (categories_direct.mean()*categories_direct.shape[0]).sort_values(ascending=False)
cate_names_direct = list(cate_counts_direct.index)

In [42]:
X = df['message'][:1000]
Y = df.iloc[:1000,4:]
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
     
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, "urlplaceholder")
    
    #Normalization 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) # Anything that isn't A through Z or 0 through 9 will be replaced by a space

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        #clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]
    return(clean_tokens)

In [6]:
X.shape

(26216,)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
#vect = TfidfVectorizer(tokenizer=tokenize)
#X_train_tfidf = vect.fit_transform(X_train)
#clf = MultiOutputClassifier(RandomForestClassifier())
#clf.fit(X_train_tfidf, Y_train)

In [7]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X.iloc[:1000], Y.iloc[:1000])
# train classifier
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x0000023A76CF6550>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
def print_scores(Y_test, Y_pred):
    for i in range(36):
        print('Feature',i, ':', Y_test.columns[i])
        print(classification_report(Y_test.iloc[:,i], Y_pred[:,i]))
        print("_______________________________________________________")
        accuracy = (Y_test.values == Y_pred).mean()
    print('Accuracy: {:.3f}'.format(accuracy))

In [10]:
Y_pred = pipeline.predict(X_test)
print_scores(Y_test, Y_pred)

Feature 0 : related
              precision    recall  f1-score   support

           0       0.67      0.09      0.16        43
           1       0.82      0.99      0.89       201
           2       0.00      0.00      0.00         6

    accuracy                           0.81       250
   macro avg       0.49      0.36      0.35       250
weighted avg       0.77      0.81      0.75       250

_______________________________________________________
Feature 1 : request
              precision    recall  f1-score   support

           0       0.88      0.63      0.73       115
           1       0.74      0.93      0.83       135

    accuracy                           0.79       250
   macro avg       0.81      0.78      0.78       250
weighted avg       0.81      0.79      0.78       250

_______________________________________________________
Feature 2 : offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       249
           1 

C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\

Feature 12 : shelter
              precision    recall  f1-score   support

           0       0.89      0.99      0.94       216
           1       0.78      0.21      0.33        34

    accuracy                           0.88       250
   macro avg       0.83      0.60      0.63       250
weighted avg       0.87      0.88      0.85       250

_______________________________________________________
Feature 13 : clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       247
           1       0.00      0.00      0.00         3

    accuracy                           0.99       250
   macro avg       0.49      0.50      0.50       250
weighted avg       0.98      0.99      0.98       250

_______________________________________________________
Feature 14 : money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       245
           1       0.00      0.00      0.00         5

    accu

C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\

Feature 22 : electricity
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       248
           1       0.00      0.00      0.00         2

    accuracy                           0.99       250
   macro avg       0.50      0.50      0.50       250
weighted avg       0.98      0.99      0.99       250

_______________________________________________________
Feature 23 : tools
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       249
           1       0.00      0.00      0.00         1

    accuracy                           1.00       250
   macro avg       0.50      0.50      0.50       250
weighted avg       0.99      1.00      0.99       250

_______________________________________________________
Feature 24 : hospitals
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       246
           1       0.00      0.00      0.00         4

   

C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\

### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__min_samples_split': [2, 3, 4],
    'clf__estimator__n_estimators': [50, 100]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv = 3, n_jobs=4, verbose = 2)
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x0000023A76CF6550>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=4,
             param_grid={'clf__estimator__min_samples_split': [2, 3, 4],
                         'clf__estimator__n_estimators': [50, 100],
                         'vect__ngram_range': ((1, 1), (1, 2))},
             verbose=2)

In [19]:
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 50, 'vect__ngram_range': (1, 1)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
Y_pred = cv.predict(X_test)
print_scores(Y_test, Y_pred)


Feature 0 : related
              precision    recall  f1-score   support

           0       0.50      0.07      0.12        43
           1       0.81      0.99      0.89       201
           2       0.00      0.00      0.00         6

    accuracy                           0.80       250
   macro avg       0.44      0.35      0.34       250
weighted avg       0.74      0.80      0.74       250

_______________________________________________________
Feature 1 : request
              precision    recall  f1-score   support

           0       0.85      0.65      0.74       115
           1       0.75      0.90      0.82       135

    accuracy                           0.79       250
   macro avg       0.80      0.78      0.78       250
weighted avg       0.80      0.79      0.78       250

_______________________________________________________
Feature 2 : offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       249
           1 

C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\

Feature 27 : other_infrastructure
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       244
           1       0.00      0.00      0.00         6

    accuracy                           0.98       250
   macro avg       0.49      0.50      0.49       250
weighted avg       0.95      0.98      0.96       250

_______________________________________________________
Feature 28 : weather_related
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       236
           1       1.00      0.07      0.13        14

    accuracy                           0.95       250
   macro avg       0.97      0.54      0.55       250
weighted avg       0.95      0.95      0.93       250

_______________________________________________________
Feature 29 : floods
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       246
           1       0.00      0.00      0.0

C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\

In [34]:
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred]), target_names = Y_test.columns[1:]))

                        precision    recall  f1-score   support

               request       0.75      0.90      0.82       135
                 offer       0.00      0.00      0.00         1
           aid_related       0.72      0.95      0.82       143
          medical_help       1.00      0.03      0.06        32
      medical_products       0.00      0.00      0.00        19
     search_and_rescue       0.00      0.00      0.00         9
              security       0.00      0.00      0.00         5
              military       0.00      0.00      0.00         3
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.69      0.82        55
                  food       0.90      0.85      0.87        71
               shelter       0.92      0.35      0.51        34
              clothing       0.00      0.00      0.00         3
                 money       0.00      0.00      0.00         5
        missing_people       0.00      

C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classif

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [35]:
 
pipeline2 = Pipeline([
     ('features', FeatureUnion([
         ('vect', TfidfVectorizer(tokenizer=tokenize)),
     ])),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
     ])
pipeline2.get_params()


{'memory': None,
 'steps': [('features', FeatureUnion(transformer_list=[('vect',
                                   TfidfVectorizer(tokenizer=<function tokenize at 0x0000023A76CF6550>))])),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'features': FeatureUnion(transformer_list=[('vect',
                                 TfidfVectorizer(tokenizer=<function tokenize at 0x0000023A76CF6550>))]),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'features__n_jobs': None,
 'features__transformer_list': [('vect',
   TfidfVectorizer(tokenizer=<function tokenize at 0x0000023A76CF6550>))],
 'features__transformer_weights': None,
 'features__verbose': False,
 'features__vect': TfidfVectorizer(tokenizer=<function tokenize at 0x0000023A76CF6550>),
 'features__vect__analyzer': 'word',
 'features__vect__binary': False,
 'features__vect__decode_error': 'strict',
 'features__vect__dtype': numpy.float64,
 'features__vect__encoding': 'utf-8',
 'featur

In [36]:
parameters = {
    'features__vect__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__n_estimators': [50, 100]
}

cv2 = GridSearchCV(pipeline2, param_grid=parameters, cv = 3, n_jobs=4, verbose = 2)
cv2.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('vect',
                                                                        TfidfVectorizer(tokenizer=<function tokenize at 0x0000023A76CF6550>))])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=4,
             param_grid={'clf__estimator__n_estimators': [50, 100],
                         'features__vect__ngram_range': ((1, 1), (1, 2))},
             verbose=2)

In [37]:
print("\nBest Parameters:", cv2.best_params_)
Y_pred = cv2.predict(X_test)
print_scores(Y_test, Y_pred)


Best Parameters: {'clf__estimator__n_estimators': 100, 'features__vect__ngram_range': (1, 1)}
Feature 0 : related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        43
           1       0.80      1.00      0.89       201
           2       0.00      0.00      0.00         6

    accuracy                           0.80       250
   macro avg       0.27      0.33      0.30       250
weighted avg       0.65      0.80      0.72       250

_______________________________________________________
Feature 1 : request
              precision    recall  f1-score   support

           0       0.75      0.69      0.71       115
           1       0.75      0.80      0.77       135

    accuracy                           0.75       250
   macro avg       0.75      0.74      0.74       250
weighted avg       0.75      0.75      0.75       250

_______________________________________________________
Feature 2 : offer
              precision    

C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       239
           1       0.62      0.45      0.53        11

    accuracy                           0.96       250
   macro avg       0.80      0.72      0.75       250
weighted avg       0.96      0.96      0.96       250

_______________________________________________________
Feature 22 : electricity
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       248
           1       0.00      0.00      0.00         2

    accuracy                           0.99       250
   macro avg       0.50      0.50      0.50       250
weighted avg       0.98      0.99      0.99       250

_______________________________________________________
Feature 23 : tools
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       249
           1       0.00      0.00      0.00         1

    accuracy              

C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vivi\anaconda3\lib\site-packages\sklearn\metrics\

### 9. Export your model as a pickle file

In [40]:
model = pipeline
pickle.dump(model, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
#Supporting pipeline
# import packages
import sys


def load_data(data_file):
    # read in file


    # clean data


    # load to database


    # define features and label arrays


    return X, y


def build_model():
    # text processing and model pipeline


    # define parameters for GridSearchCV


    # create gridsearch object and return as final model pipeline


    return model_pipeline


def train(X, y, model):
    # train test split


    # fit model


    # output model test results


    return model


def export_model(model):
    # Export model as a pickle file



def run_pipeline(data_file):
    X, y = load_data(data_file)  # run ETL pipeline
    model = build_model()  # build model pipeline
    model = train(X, y, model)  # train model pipeline
    export_model(model)  # save model


if __name__ == '__main__':
    data_file = sys.argv[1]  # get filename of dataset
    run_pipeline(data_file)  # run data pipeline